# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
#import datetime
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3, botocore
import pandas as pd
import numpy as np

In [ ]:
!pip install botocore --upgrade
!pip install boto3 --upgrade

Requirement already up-to-date: botocore in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (1.19.62)


In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Add New items to Catalog <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [4]:
#original_data = pd.read_csv(dataset_dir + '/movies.csv', sep=',', encoding='latin-1', dtype={'movieId': "int64", 'title': "str"})
original_data = pd.read_csv(dataset_dir + '/movies.csv')
original_data.tail(5)

,movieId,title,genres
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


As you can see there are (9742 for small) titles in the dataset, we will add a new dataframe for creation timestamp. If you don’t provide the CREATION_TIMESTAMP, the model infers this information from the interaction dataset and uses the timestamp of the item’s earliest interaction as its corresponding release date. If an item doesn’t have an interaction, its release date is set as the timestamp of the latest interaction in the training set and it is considered a new item with age 0.

In [105]:
original_data['CREATION_TIMESTAMP'] = 0

In [70]:
import datetime
new_titles = {'movieId': ['193618', '193619', '193620', '193621'], 'title': ['NewTitle9' + "(" + str(datetime.datetime.now().year) + ")", 'NewTitle10' + "(" + str(datetime.datetime.now().year) + ")", 'NewTitle11' + "(" + str(datetime.datetime.now().year) + ")", 'NewTitle12' + "(" + str(datetime.datetime.now().year) + ")"],'genres': ['Action|Comedy', 'Sci-Fi|Fantasy', 'Drama|Thriller', 'Documentary|IMAX'],'CREATION_TIMESTAMP': [round(time.time()) - 1000, round(time.time()) - 2000, round(time.time()) - 3000, round(time.time())]}
new_titles_df = pd.DataFrame(new_titles)
new_titles_df

,movieId,title,genres,CREATION_TIMESTAMP
0,193618,NewTitle9(2020),Action|Comedy,1603374049
1,193619,NewTitle10(2020),Sci-Fi|Fantasy,1603373049
2,193620,NewTitle11(2020),Drama|Thriller,1603372049
3,193621,NewTitle12(2020),Documentary|IMAX,1603375049


In [69]:
print(new_titles)

{'movieId': ['193614', '193615', '193616', '193617'], 'title': ['NewTitle5(2020)', 'NewTitle6(2020)', 'NewTitle7(2020)', 'NewTitle8(2020)'], 'genres': ['Action|Comedy', 'Sci-Fi|Thriller', 'Drama', 'Documentary'], 'CREATION_TIMESTAMP': [1600435943, 1600434943, 1600433943, 1600436943]}


In [96]:
for title in new_titles_df:
    print(new_titles['movieId'][index])
    

NameError: name 'index' is not defined

In [99]:
print (round(time.time()) - 1000)

1603378354


In [109]:
currenttime = round(time.time()) - 1000
currentyear = datetime.utcfromtimestamp(currenttime).strftime('%Y')

In [110]:
print(currenttime)
print(currentyear)
print(datetime.utcfromtimestamp(currenttime).strftime('%Y-%m-%d %H:%M:%S'))

1611943602
2021
2021-01-29 18:06:42


AttributeError: module 'boto3' has no attribute 'Version'

In [137]:
personalize_events.put_items(
    datasetArn = items_dataset_arn,
    items = {
        'itemId': 193620,
        'properties': {"GENRE": "NewTitle13", "GENRE": "Action|Comedy", "CREATION_TIMESTAMP": currenttime, "YEAR": currentyear
                      }
    })

AttributeError: 'PersonalizeEvents' object has no attribute 'put_items'

now we will add the titles to item metadata

In [71]:
#original_data = original_data.append(new_titles_df, ignore_index=True)
#original_data.tail(10)

lets confirm the CREATION_TIMESTAMP is in the correct format

In [9]:
from datetime import datetime
creation_time_stamp = original_data.iloc[-1]['CREATION_TIMESTAMP']
print(creation_time_stamp)
print(datetime.utcfromtimestamp(creation_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1600436943
2020-09-18 13:49:03


Now we will save this csv to use in the same way we used the original movies.csv

In [10]:
original_data.to_csv((dataset_dir+"/"+"movies.csv"), index=False, float_format='%.0f')

here we will go through the same process as in '02_Validating_and_Importing_Item_Metadata.ipynb'

In [11]:
original_data['year'] =original_data['title'].str.extract('.*\((.*)\).*',expand = False)
original_data.head(5)

,movieId,title,genres,CREATION_TIMESTAMP,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,0,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,1995
4,5,Father of the Bride Part II (1995),Comedy,0,1995


In [12]:
itemmetadata_df = original_data.copy()
itemmetadata_df = itemmetadata_df[['movieId', 'genres', 'year','CREATION_TIMESTAMP']]
itemmetadata_df.head()

,movieId,genres,year,CREATION_TIMESTAMP
0,1,Adventure|Animation|Children|Comedy|Fantasy,1995,0
1,2,Adventure|Children|Fantasy,1995,0
2,3,Comedy|Romance,1995,0
3,4,Comedy|Drama|Romance,1995,0
4,5,Comedy,1995,0


In [13]:
itemmetadata_df.rename(columns = {'genres':'GENRE', 'movieId':'ITEM_ID', 'year':'YEAR'}, inplace = True) 

Now we will storee the item-metadata for importing the movies with the new titles

In [14]:
itemmetadata_filename = "item-meta.csv"
itemmetadata_df.to_csv((data_dir+"/"+itemmetadata_filename), index=False, float_format='%.0f')

Upload data to S3

Now that your the item-metadata has been updated with the new titles, upload the CSV file of our new item-metadata.

In [15]:
itemmetadata_file_path = data_dir + "/" + itemmetadata_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(itemmetadata_filename).upload_file(itemmetadata_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+itemmetadata_filename

Now we will import the updated item-meta.csv in to the "personalize-poc-movielens-items". 

In [16]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-item-" + "-" + str(round(time.time()*1000)),
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, itemmetadata_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:051545784337:dataset-import-job/personalize-poc-item--1600436984815",
  "ResponseMetadata": {
    "RequestId": "c407d0d4-5c05-48e0-915a-307738a19fdd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 18 Sep 2020 13:49:44 GMT",
      "x-amzn-requestid": "c407d0d4-5c05-48e0-915a-307738a19fdd",
      "content-length": "123",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Before we can update the campaign, the import job must be active. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the import job every second, up to a maximum of 6 hours. This should take approximately 15-20 mins

In [17]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 81 ms, sys: 7.21 ms, total: 88.2 ms
Wall time: 14min 1s


In [18]:
user_personalization_update_solution_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn, 
    trainingMode='UPDATE')
new_user_personalization_solution_version_arn = user_personalization_update_solution_response['solutionVersionArn']
print("Creating solution version: {}".format(new_user_personalization_solution_version_arn))


Creating solution version: arn:aws:personalize:us-east-1:051545784337:solution/personalize-poc-userpersonalization/d25637a8


To update the model(solutionVersion), we ran the createSolutionVersion with trainingMode set to UPDATE. This updates the model with the latest item information and adjusts the exploration according to implicit feedback from the users. This is not equivalent to training a model, which you can do by setting trainingMode to FULL. You should perform full training less frequently, typically once every 1–5 days. When the new updated solutionVersion is created, you can update the campaign to get recommendations using it.

In [ ]:
status = None
max_time = time.time() + 60*60 # 1 hour
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
    solutionVersionArn = new_user_personalization_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))

    if status == "ACTIVE" or status == "CREATE FAILED":
         break
    time.sleep(60)

Now that the solution is updated with the new solution version, including item-metadata information about our new titles, we can update the campaign to use this version. You now have the option of setting additional configuration for the campaign, which allows you to adjust the exploration Amazon Personalize does for the item recommendations and therefore adjust the results. These settings are only available if you’re creating a campaign whose solution version uses the user-personalization recipe. The configuration options are as follows:

    explorationWeight – Higher values for explorationWeight signify higher exploration; new items with low impressions are more likely to be recommended. A value of 0 signifies that there is no exploration and results are ranked according to relevance. You can set this parameter in a range of [0,1] and its default value is 0.3.
    
    explorationItemAgeCutoff – This is the maximum duration in days relative to the latest interaction(event) timestamp in the training data. For example, if you set explorationItemAgeCutoff to 7, the items with an age over or equal to 7 days aren’t considered cold items and there is no exploration on these items. You may still see some items older than or equal to 7 days in the recommendation list because they’re relevant to the user’s interests and are of good quality even without the help of the exploration. The default value for this parameter is 30, and you can set it to any value over 0.

In [32]:
userpersonalization_update_campaign_response = personalize.update_campaign(campaignArn=userpersonalization_campaign_arn, solutionVersionArn=new_user_personalization_solution_version_arn)
userpersonalization_campaign_arn = userpersonalization_update_campaign_response['campaignArn']
print(json.dumps(userpersonalization_update_campaign_response, indent=2))

ResourceInUseException: An error occurred (ResourceInUseException) when calling the UpdateCampaign operation: Campaign is already being updated

In [34]:
describe_campaign_update_response = personalize.describe_campaign(
            campaignArn = userpersonalization_campaign_arn)
status = describe_campaign_update_response["campaign"]["status"]
print(status)
print(describe_campaign_update_response)

ACTIVE
{'campaign': {'name': 'personalize-poc-userpersonalization', 'campaignArn': 'arn:aws:personalize:us-east-1:051545784337:campaign/personalize-poc-userpersonalization', 'solutionVersionArn': 'arn:aws:personalize:us-east-1:051545784337:solution/personalize-poc-userpersonalization/5a95063f', 'minProvisionedTPS': 1, 'campaignConfig': {}, 'status': 'ACTIVE', 'creationDateTime': datetime.datetime(2020, 9, 17, 23, 56, 1, 565000, tzinfo=tzlocal()), 'lastUpdatedDateTime': datetime.datetime(2020, 9, 18, 14, 48, 16, 729000, tzinfo=tzlocal()), 'latestCampaignUpdate': {'solutionVersionArn': 'arn:aws:personalize:us-east-1:051545784337:solution/personalize-poc-userpersonalization/d25637a8', 'status': 'CREATE IN_PROGRESS', 'creationDateTime': datetime.datetime(2020, 9, 18, 14, 47, 55, 504000, tzinfo=tzlocal()), 'lastUpdatedDateTime': datetime.datetime(2020, 9, 18, 14, 48, 17, 115000, tzinfo=tzlocal())}}, 'ResponseMetadata': {'RequestId': 'dc074094-d4db-48c0-a9b8-ce1ce8dec510', 'HTTPStatusCode': 

In [ ]:
status = None
max_time = time.time() + 60*60 # 1 hour
while time.time() < max_time:
    describe_campaign_update_response = personalize.describe_campaign(
            campaignArn = userpersonalization_campaign_arn
        )
    status = describe_campaign_update_response["campaign"]["status"]
    print("CampaignUpdate: {}".format(status))

    if status == "ACTIVE" or status == "CREATE FAILED":
         break
    time.sleep(60)

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
newitems_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
newitems_df.head(5)

We will create a movie lookup function as in the '05_Interacting_with_Campaigns_and_Filters.ipynb' notebook

In [ ]:
def get_movie_by_id(movie_id, movie_df=newitems_df):
    """
    This takes in an movie_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [35]:
# A known good id (NewTitle5)
print(get_movie_by_id(movie_id="193614"))
# A bad type of value
print(get_movie_by_id(movie_id='987.9393939'))
# Really bad values
print(get_movie_by_id(movie_id='Steve'))

NewTitle5(2020)
Error obtaining title
Error obtaining title


Great! Now we have a way of rendering results. 

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [36]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[410, 307, 413]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [54]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [55]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,410,307,413
0,Get Out (2017),Get Out (2017),Get Out (2017)
1,"Silence of the Lambs, The (1991)",It (2017),It (2017)
2,"Shining, The (1980)",Planet Terror (2007),NewTitle6(2020)
3,"Godfather, The (1972)","Shining, The (1980)",Traffic (2000)
4,Baby Driver (2017),Grindhouse (2007),Ocean's Eleven (2001)
5,Spotlight (2015),Zombieland (2009),10 Cloverfield Lane (2016)
6,NewTitle6(2020),Shaun of the Dead (2004),Interstellar (2014)
7,Traffic (2000),Annabelle (2014),Spotlight (2015)
8,"Big Short, The (2015)",Monty Python's The Meaning of Life (1983),The Salt of the Earth (2014)
9,Rogue One: A Star Wars Story (2016),No Country for Old Men (2007),The Nice Guys (2016)


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

In [44]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [50]:
TRACKING_ID = '592baa2a-31a9-4739-8586-89529a8cd052'

In [53]:
# Note this will take about 15 seconds to complete due to the sleeps
for user in users:
    for newtitle in newtitles
    event_type = 'click'
    user_id = user
    movie = '193614'
    send_movie_click(user_id, movie,'click')
    send_movie_click(user_id, movie,'watch')
    print('sending event ' + event_type + ' for ' + get_movie_by_id(movie))
    movie = '193615'
    send_movie_click(user_id, movie,'click')
    send_movie_click(user_id, movie,'watch')
    print('sending event ' + event_type + ' for ' + get_movie_by_id(movie))
    movie = '193616'
    send_movie_click(user_id, movie,'click')
    send_movie_click(user_id, movie,'watch')
    print('sending event ' + event_type + ' for ' + get_movie_by_id(movie))
    movie = '193617'
    send_movie_click(user_id, movie,'click')
    send_movie_click(user_id, movie,'watch')
    print('sending event ' + event_type + ' for ' + get_movie_by_id(movie))
    time.sleep(5)

sending event click for NewTitle5(2020)
sending event click for NewTitle6(2020)
sending event click for NewTitle7(2020)
sending event click for NewTitle8(2020)
sending event click for NewTitle5(2020)
sending event click for NewTitle6(2020)
sending event click for NewTitle7(2020)
sending event click for NewTitle8(2020)
sending event click for NewTitle5(2020)
sending event click for NewTitle6(2020)
sending event click for NewTitle7(2020)
sending event click for NewTitle8(2020)


In [ ]:
for movienew_titles

Now lets apply item filters to see recommendations for one of these users within a genre


In [56]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

You can see the recommendations for movies within a given genre. Within a VOD application you could create Shelves (also known as rails or carosels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

In [80]:
recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)
for filter_arn in decade_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)

recommendations_df_shelves

,Crime,IMAX,Documentary,Fantasy,Horror,Children,Comedy,1970s
0,Traffic (2000),Interstellar (2014),The Salt of the Earth (2014),Rogue One: A Star Wars Story (2016),Get Out (2017),Moana (2016),"Honest Liar, An (2014)",Chinatown (1974)
1,Ocean's Eleven (2001),Star Wars: Episode VII - The Force Awakens (2015),"Honest Liar, An (2014)",Moana (2016),It (2017),Inside Out (2015),Get Hard (2015),"Godfather, The (1972)"
2,The Nice Guys (2016),Transcendence (2014),Won't You Be My Neighbor? (2018),The Dark Tower (2017),"Shining, The (1980)",WALLÂ·E (2008),Moana (2016),"Clockwork Orange, A (1971)"
3,Nightcrawler (2014),Captain America: The Winter Soldier (2014),Going Clear: Scientology and the Prison of Bel...,Sorry to Bother You (2018),Victor Frankenstein (2015),Home (2015),Snatch (2000),Alien (1979)
4,Chinatown (1974),RoboCop (2014),Citizenfour (2014),Bright (2017),"Silence of the Lambs, The (1991)","Incredibles, The (2004)",Sorry to Bother You (2018),Taxi Driver (1976)
5,"Godfather, The (1972)",I Am Legend (2007),Making a Murderer (2015),Inside Out (2015),Black Mirror: White Christmas (2014),Paddington 2 (2017),A Million Ways to Die in the West (2014),Blazing Saddles (1974)
6,"Silence of the Lambs, The (1991)",Divergent (2014),Bowling for Columbine (2002),Star Wars: Episode VII - The Force Awakens (2015),The Dark Tower (2017),Minions (2015),Horrible Bosses 2 (2014),"Exorcist, The (1973)"
7,Hell or High Water (2016),Edge of Tomorrow (2014),Quest (2017),Star Wars: The Last Jedi (2017),The Cloverfield Paradox (2018),Sapphire Blue (2014),Inside Out (2015),Star Wars: Episode IV - A New Hope (1977)
8,Baby Driver (2017),Need for Speed (2014),How To Change The World (2015),The Jungle Book (2016),Annabelle (2014),Kubo and the Two Strings (2016),Merci Patron ! (2016),Paper Moon (1973)
9,Suicide Squad (2016),Noah (2014),Planet Earth II (2016),X-Men: Apocalypse (2016),Knock Knock (2015),Paddington (2014),Focus (2015),Invasion of the Body Snatchers (1978)


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `06_Clean_Up_Resources.ipynb`.

In [ ]:
%store event_tracker_arn
%store batchInferenceJobArn